In [1]:
import pandas as pd
import sklearn as sk

In [ ]:
def loss_calc(df):
    #for each position
        #load in fantasypros preseason rankings for one position
        #add in a column at beginning with preseason rank by using order of rows
        #load in projections from best model (check github for these)
        #load in fantasy actual results
        #create table to store loss statistics
        #calculate loss metrics against Fantasy Football Rankings
        #store in stats table
        #calculate loss metrics against own model predictions
        #store in stats table
        #print stats table
        #return stats table
        prediction_results = df
        return prediction_results
